# Intermediate interface 

## Aim of this is notebook is to create an interface that can help create key between two user

## Explanation
    * Alice will generate a bases (Alice will create)
    * Alice send the bases to the interface 
    * Bob will generate a bases (Bob will create)
    * Bob send the bases to the interface    
    * Interface now generate a random initial states and measure it with Alice and Bob bases
    * Interface return the measured value to Alice and Bob
    * Alice and Bob annouce their bases through MQTT server 
    * They create a key from the measured value by comparing the bases and eliminate the garbage bits
    * After having the key, they can create encrypted msg with the key and send to each other

# Intermediate Inteface  main function

In [3]:
import numpy as np
import os

from qiskit.tools.visualization import circuit_drawer, plot_histogram
from qiskit import execute, QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import BasicAer
from IPython.display import display, Math, Latex

from quantuminspire.credentials import get_authentication
from quantuminspire.api import QuantumInspireAPI
from quantuminspire.qiskit import QI

QI_URL = os.getenv('183033b2e8fe77a4c598e44fdc1a0c9486bd1683', 'https://api.quantum-inspire.com/')

In [4]:
import numpy as np
from qiskit import*

## Input value to the intermediate interface (these cells are here to test the main function of interface only)

In [5]:
def SenderA_random_bases(size):
    np.random.seed(size)
    random_bases = np.random.randint(2, size=size) # Here we let bases = 0,1 corresponding to bases X,Z
    return random_bases

In [6]:
def SenderB_random_bases(size):
    np.random.seed(size+1)
    random_bases = np.random.randint(2, size=size) # Here we let bases = 0,1 corresponding to bases X,Z
    return random_bases

## Intermediate interface inner function

In [7]:
#### Create initial random state ###########################################################
def Inteface_intial_random_state(size, seed):
    np.random.seed(seed)
    random_msg = np.random.randint(4, size=size)
    return random_msg

In [8]:
def Intial_state_generation(qc,state,index):
    if(state == 0):
        pass
    elif(state == 1):
        qc.x(index)
    elif(state == 2):
        qc.h(index)
    else:
        qc.x(index)
        qc.h(index)

In [9]:
def Measurement_generation(qc,measure,index):
    if(measure == 0):
        qc.h(index)

In [10]:
#### Create quantum circuits with according intial states and measurement bases ###########################################################
def create_qc_measurement_bases(intial_state, base):
    qc = QuantumCircuit(2,2)
    for i in range(len(intial_state)):
        Intial_state_generation(qc,intial_state[i],i)
    for i in range(len(base)):
        Measurement_generation(qc,base[i],i)
    qc.draw()
    return qc

In [11]:
#### Intialize the random state, combine the bases, create the quantum circuits #############################################################
def Intermediate_interface(SenderA_bases, SenderB_bases, size_of_message, number_of_user):
    intial_state = Inteface_intial_random_state(2*size_of_message, 2*size_of_message+1)
    print(intial_state)
    bases_combination = [*SenderA_bases,*SenderB_bases]
    print(bases_combination)
    encrypted_message = []
    for i in range(size_of_message):
        qc = create_qc_measurement_bases([intial_state[i],intial_state[i+size_of_message]],[bases_combination[i],bases_combination[i+size_of_message]])
        for i in range(number_of_user):
            qc.measure(i,i)
        encrypted_message.append(qc)
    return encrypted_message

In [12]:
##### Intermediate Interface send job to Quantum Inspire ############################################################
def run_quantum_inspire(encrypted_message):
    bases = Inteface_intial_random_state(len(encrypted_message),len(encrypted_message)+2)
    qi_backend = QI.get_backend('QX-34-L')
    j = execute(encrypted_message, backend=qi_backend, shots=512)
    result = j.result()
    return result

In [13]:
##### Parsing the result from the Quantum Inspire ############################################################
def return_message(counts):
    message_A = [0]*len(counts)
    message_B = [0]*len(counts)
    for i in range (len(counts)):
        key_list = list(count[i].keys())
        for j in key_list:
            if message_A[i] == 0:
                message_A[i] = j[0]
            elif ((message_A[i] != 0) & (message_A[i] != j[0])):
                message_A[i] = 'h'
            
            if message_B[i] == 0:
                message_B[i] = j[1]
            elif ((message_B[i] != 0) & (message_B[i] != j[1])):
                message_B[i] = 'h'
    return message_A, message_B

## Example of what Intermediate interface will do

In [14]:
size_of_message = 5
number_of_user = 2

In [15]:
senA = SenderA_random_bases(size_of_message)
print(senA)

[1 0 1 1 0]


In [16]:
senB = SenderB_random_bases(size_of_message)
print(senB)

[0 1 1 0 0]


In [17]:
encrypt = Intermediate_interface(senA,senB,size_of_message,number_of_user)

[1 3 0 3 1 3 1 0 1 3]
[1, 0, 1, 1, 0, 0, 1, 1, 0, 0]


In [31]:
encrypt[0].draw()

┌───┐     ┌─┐        
q_0: ┤ X ├─────┤M├────────
     ├───┤┌───┐└╥┘┌───┐┌─┐
q_1: ┤ X ├┤ H ├─╫─┤ H ├┤M├
     └───┘└───┘ ║ └───┘└╥┘
c: 2/═══════════╩═══════╩═
                0       1

In [19]:
authentication = get_authentication()
QI.set_authentication(authentication, QI_URL)

Enter email:
bao.bachbbace12@hcmut.edu.vn
Enter password
········


In [32]:
result = run_quantum_inspire(encrypt)
count = result.get_counts()

In [33]:
count

[{'11': 512},
 {'11': 512},
 {'00': 512},
 {'00': 145, '01': 126, '10': 118, '11': 123},
 {'10': 254, '11': 258}]

In [34]:
msg_a,msgb = return_message(count)

In [35]:
msg_a

['1', '1', '0', 'h', '1']

In [36]:
msgb

['1', '1', '0', 'h', 'h']

# Transmitting protocol
### We are using MQTT protocol which can give us the ability to subcribe to a feed. By subcribing to the feed, we will receive the data whenever there is a new data that is published to that feed the clear idea is written in readme file

In [37]:
import sys
from Adafruit_IO import MQTTClient
import random
import time
import serial.tools.list_ports

In [38]:
AIO_FEED_ID_1 = "intermediate-interface"
AIO_FEED_ID_A = "user-a"
AIO_FEED_ID_B = "user-b"
AIO_USERNAME = "bbace"
AIO_KEY = "aio_bPYP32NOKkrTiko7nYSwDr9YlTfz"

In [39]:
##### Process data from the feed then save the bases ############################################################
def processData(data):
    global A_bases_buffer
    global B_bases_buffer
    global size_of_msg
    splitData = data.split(":")
    if(splitData[0] == 'UserA'):
        size_of_msg = int(splitData[1])
        c = 1
        for i in range(size_of_msg):
            A_bases_buffer.append(int(splitData[2][i+c]))
            c += 1
    elif(splitData[0] == 'UserB'):
        size_of_msg = int(splitData[1])
        c = 1
        for i in range(size_of_msg):
            B_bases_buffer.append(int(splitData[2][i+c]))
            c += 1

In [40]:
def connected( client ):
    print ("Successfully connected ...")
    client.subscribe ("intermediate-interface")
def subscribe( client , userdata , mid , granted_qos ):
    print(" Subcribe sucessfully...")

def disconnected( client ):
    print(" Disconnected...")
    sys.exit (1)
##### Whenever there is a new data in feed, this function will call ############################################################
def message(client , feed_id , payload):
    global count
    global A_bases_buffer
    global B_bases_buffer
    print("Received data: "+ feed_id +" "+payload)
    count = count + 1
    processData(payload)
#     print(A_bases_buffer)
#     print(B_bases_buffer)
#     print(count)

In [41]:
A_bases_buffer = []
B_bases_buffer = []
size_of_msg = 0
count = 0

## Main process of the MQTT protocol using Adafruit server to implement QKD
### The process tend to run 24/7 and always checking for the data

In [42]:
client = MQTTClient(AIO_USERNAME , AIO_KEY)
client.on_connect    = connected
client.on_disconnect = disconnected
client.on_message    = message
client.on_subscribe  = subscribe
client.connect()
client.loop_background()
while True :
############## When already get the data of all user, start the function of intermediate interface then return result#################
    if(count == number_of_user): 
        encrypt = Intermediate_interface(A_bases_buffer,B_bases_buffer,size_of_msg,number_of_user)
        result  = run_quantum_inspire(encrypt)
        count = result.get_counts()
        msg_a,msg_b = return_message(count)
        print(msg_a)
        print(msg_b)
        client.publish(AIO_FEED_ID_A, "UserA_result:"+str(size_of_msg)+':'+str(msg_a))
        client.publish(AIO_FEED_ID_B, "UserB_result:"+str(size_of_msg)+':'+str(msg_b))
        
        ## Restart the buffer
        A_bases_buffer = []
        B_bases_buffer = []
        size_of_msg = 0
        count = 0

Connected to Adafruit IO!
Successfully connected ...
 Subcribe sucessfully...
Received data: intermediate-interface UserA:5:[0 1 1 0 1]
Received data: intermediate-interface UserB:5:[1 1 0 0 1]
[1 3 0 3 1 3 1 0 1 3]
[0, 1, 1, 0, 1, 1, 1, 0, 0, 1]
['h', '1', 'h', 'h', 'h']
['h', 'h', '0', '1', '1']


KeyboardInterrupt: 